# Mentoría Ciencia de datos aplicada a la predicción de licencias médicas y comportamiento de los colaboradores

# Trabajo Práctico 2: Análisis Exploratorio y Curación

Trabajar con el csv denominado data.csv y resolver las siguientes consignas:

1) Analisis e imputación de valores faltantes, nulos y/o ruidosos en las columnas relevantes del dataset. 

    1.A) ¿Existe alguna relación entre los datos faltantes? Usar la librería missingno para resolver esta consigna.
    1.B) Realizar la imputación de aquellas columnas con datos faltantes, justificar el método.
    1.C) Trabajar con las columnas dias_aprobados y dias_solicitados, observar si existe coherencia en las mismas y datos nulos o ruidosos que deban imputar. 

2) Observar si existen outliers en las columnas relevantes. Tomar una decisión con los mismos y justificarla. 

3) Trabajar con las columnas categóricas del dataset, decidir si corresponde realizar algún reagrupamiento, considerando su frecuencia. 
   Ayuda: tener en cuenta la presencia de caracteres especiales y/o espacios vacíos en la columna diagnostico.
   
   3.1) Para el reagrupamiento de la columna diagnostico, ayudarse de la base externa, https://icdcode.info/espanol/cie-10/codigos.html que contiene la clasificación y agrupación de diagnosticos a nivel internacional. Les dejamos el csv de la misma y una breve explicación de como utilizarla https://raw.githubusercontent.com/verasativa/CIE-10/master/cie-10.csv. 
   Para la union de esta base externa con el dataframe de licencias, deberan usar el data_key.csv que le adjuntamos en este TP. 
   
4) A partir de la columna fecha_creacion, generar dos columnas extras temporales utilizando las escalas temporales que consideren adecuadas, por ejemplo: día, semana, quincena, mes, trimestre o semestre. Pueden utilizar la función resample de pandas. 

5) Generar y guardar el nuevo dataset con todas las tranformaciones realizadas, el cual sera el input de los siguientes tp de aprendizaje supervisado/no supervisado. 

## Carga del dataset

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

url = 'https://raw.githubusercontent.com/MentoriaDiploDatos/Presentacion/main/data.csv'
df = pd.read_csv(url, sep=';')

In [2]:
df.head(2)

,numero_licencia,fecha_creacion,dispositivo_acceso,dias_solicitados,fecha_inicio,fecha_fin,diagnostico,id_diagnostico,id_usuario,fecha_nacimiento,genero,tipo_licencia,estado_civil,dias_aprobados,decision_medica,id_auditor,motivo_rechazo,categoria_diagnostico
0,1,2021-01-01 00:10:10.418,APP,2.0,2021-01-01,2021-01-01,EFECTOS ADVERSOS DE VACUNAS VIRALES,NaN,15508,1979-11-28,FEMALE,Médica,OTHER,2.0,EDITAR_Y_APROBAR,14721.0,NaN,NaN
1,105,2021-01-02 14:14:20.330,APP,2.0,2021-01-02,2021-01-03,EFECTOS ADVERSOS DE VACUNAS VIRALES,NaN,15508,1979-11-08,FEMALE,Médica,OTHER,2.0,EDITAR_Y_APROBAR,14721.0,NaN,NaN


In [3]:
url_key = 'https://raw.githubusercontent.com/MentoriaDiploDatos/Exploracion-Y-Curacion/main/data_key.csv'
keys = pd.read_csv(url_key, sep=';')

In [4]:
keys.sample(2)

,numero_licencia,key
260267,279837,J04
113921,284651,M54.2


## Utilizacion de la base CIE.10 de agrupadores de diagnosticos

In [5]:
df_cod =pd.read_csv('https://raw.githubusercontent.com/verasativa/CIE-10/master/cie-10.csv')

Veamos el diagnostico correspondiente a la key A02, en este caso se corresponde con la descripcion Otras infecciones debidas a Salmonella.

In [6]:
df_cod[df_cod['code'] =='A02']

,code,code_0,code_1,code_2,code_3,code_4,description,level,source
375,A02,A00-B99,A00-A09,NaN,NaN,NaN,Otras infecciones debidas a Salmonella,2,icdcode.info


A partir de los grupos y subgrupos que figuran en las columnas code_0, code_1 y siguientes, podemos obtener las descripciones correspondientes. 

Observamos que el significado del grupo A00-B99, al que pertenece A02, es Ciertas enfermedades infecciosas y parasitarias (observar que el level es 0)

In [7]:
df_cod[df_cod['code'] == df_cod[df_cod['code'] =='A02']['code_0'].values[0]]

,code,code_0,code_1,code_2,code_3,code_4,description,level,source
0,A00-B99,NaN,NaN,NaN,NaN,NaN,Ciertas enfermedades infecciosas y parasitarias,0,icdcode.info


Tambien podemos obtener el grupo siguiente al que pertenece este diagnostico, de la siguiente manera (observar que el level es 1).

In [8]:
df_cod[df_cod['code'] == df_cod[df_cod['code'] =='A02']['code_1'].values[0]]

,code,code_0,code_1,code_2,code_3,code_4,description,level,source
64,A00-A09,A00-B99,NaN,NaN,NaN,NaN,Enfermedades infecciosas intestinales,1,icdcode.info


Un ejemplo de como podrian ir incorporando las descripciones de grupos y subgrupos a cada diagnostico individual, es haciendo joins del dataframe con si mismo utilizando como clave los codigos que corresponden. 

In [9]:
df_complete_cod = df_cod.merge(df_cod[['code','description']], how='left',left_on='code_0',right_on='code')

In [10]:
df_complete_cod.rename(columns={'code_x':'code', 'description_x':'description','description_y':'description_group_level0'},inplace=True)
df_complete_cod.drop(columns=['code_y'],inplace=True)

In [11]:
df_complete_cod[df_complete_cod['code'] =='A02']

,code,code_0,code_1,code_2,code_3,code_4,description,level,source,description_group_level0
375,A02,A00-B99,A00-A09,NaN,NaN,NaN,Otras infecciones debidas a Salmonella,2,icdcode.info,Ciertas enfermedades infecciosas y parasitarias
